In [12]:
from pyclick.click_models.Evaluation import LogLikelihood, Perplexity

from pyclick.click_models.SDBN import SDBN
from pyclick.click_models.CTR import GCTR

from session_storage import SessionStorage

# на предупреждение можно не обращать внимания. Оно в assert-те и не влияет на работоспособность

In [13]:
# Простейшая обертка. Подробнее о логике работы см. в doc-string в session_storage.py
ya_storage = SessionStorage("./data/YandexRelPredChallenge.txt")
vk_storage = SessionStorage("./data/VKVideoClickSessions.txt")

Теперь можно поучить различные модели. Алгоритмы в библиотеке повторяют псевдокод из статей и книг, однако не оптимизированы.

Заметим, что можно обучать модель на непересекающихся запросах можно совершенно свободно. То есть, либо уча несколько моделей параллельно, либо исполняя параллельно непосредственно сами итерации обучения внутри одной модели, мы получили бы идентичные результаты.

Для академических задач вполне хватит и синхронных реализаций из PyClick. 

Однако, чтобы не ждать слишком долго, на семинаре мы познакомимся с достаточно "быстрыми" моделями.

На лекции говорилось, что у DBN есть параметр $\gamma$, который, по-хорошему, следует подбирать под каждый датасет.

Однако сами же авторы признают, что лучшие результаты получаются при значение $\gamma$ около 0.95, а задание $\gamma = 1$ сильно упрощает расчеты, при этом не сильно теряя в качестве.

DBN с фиксированным $\gamma = 1$ называется SDBN и работает куда быстрее. Именно такую версию мы и будем использовать

In [14]:
# Выбираем кликовую модель 
ya_model = SDBN()
# Как и другие модели обучаем на тренировочном множестве, а проверять работу -- на тестовом
ya_model.train(ya_storage.get_train_sessions())

Как видно, работает достаточно быстро для ~14k сессий.

Теперь оценим качество получившейся модели на тестовом множестве.

Для оценки качества используется перплексия. Она считается по каждой позиции:

$$
PPL@j = 2^{-\frac{1}{N}\sum_{i=1}^{N}c_{i,j}\log\mathcal{P}_{i,j} + (1 - c_{i,j})\log(1 - \mathcal{P}_{i,j}) },
$$
где $N$ -- количество запросов, $c_{i,j}$ метка фактически совершенного клика по запросу $i$ на позиции в серпе $j$, а $\mathcal{P}_{i,j}$ -- вероятность такого клика, которую предсказала наша модель

средняя перплексия по всем позициям ($M$ - длина серпа)

$$
PPL = \frac{1}{M} \cdot \sum_{j=1}^{M} PPL@j
$$

Само по себе значение перплексии о качестве модели говорит мало, но её можно использовать для сравнения моделей между собой. Чем ниже перплексия -- тем лучше

In [15]:
def eval_model(model, storage):
    log_likelihood = LogLikelihood()
    perplexity = Perplexity()

    print(f"LL\t=\t{log_likelihood.evaluate(model, storage.get_test_sessions())}")

    # Перплексия считается для каждой позиции (PPL@j)
    # Под индексом 1 лежит список перплексий по позициям
    # Под индексом 0 лежит средняя перплексия, именно по ней мы и будем сравнивать модели

    print(f"PPL\t=\t{perplexity.evaluate(model, storage.get_test_sessions())[0]}")

In [16]:
eval_model(ya_model, ya_storage)

LL	=	-0.3033771682116208
PPL	=	1.3353151793972282


Повторим аналогичную процедуру на датасете VK

In [17]:
vk_model = SDBN()
vk_model.train(vk_storage.get_train_sessions())

eval_model(vk_model, vk_storage)

LL	=	-0.21916613507449373
PPL	=	1.2893631037811262


Как видно, у модели на датасете VK PPL получилось меньше, LL больше.

Значит ли это, что она лучше?

Обучим простейшую CTR-based модель, которая была первой на лекции.

Модель основана на простом подсчете доли кликов в документ, т.е. расчет этой доли на тренировочном множестве дает нам оценку вероятности клика на документ по запросу

In [18]:
ya_ctr_model = GCTR()
ya_ctr_model.train(ya_storage.get_train_sessions())

eval_model(ya_ctr_model, ya_storage)

LL	=	-0.3829820592200756
PPL	=	1.5553359036805376


Качество существенно отличается в худшую сторону, в том числе и для VK



In [19]:
vk_ctr_model = GCTR()
vk_ctr_model.train(vk_storage.get_train_sessions())

eval_model(vk_ctr_model, vk_storage)

LL	=	-0.41639661047756865
PPL	=	1.6643166426006182


Отлично, у нас есть обученная модель, которая несколько лучше других предсказывает вероятность клика в документ по запросу. Как нам получить эту вероятность для конкретного документа по конкретному запросу?

In [20]:
# Возьмем какой-нибудь запрос, для этого можно заглянуть в storage.get_test_queries() и выбрать тот, который больше нравится

query = '102845'

# А также нам понадобится какой-нибудь документ.
# Возьмем несколько документов:
# самый закликанный. модель, очевидно, должна считать его релевантным
relevant_document = vk_storage.get_document(query=query, is_clicked=True)

# наименее закликанный, но из того же серпа, то есть менее релевантный
irrelevant_document = vk_storage.get_document(query=query, is_clicked=False)

# и посмотрим на оценки, которые модель дает таким документам по данному запросу

print(f"relevant\t(id {relevant_document})\t=\t{vk_model.predict_relevance(query, relevant_document)}")
print(f"irrelevant\t(id {irrelevant_document})\t=\t{vk_model.predict_relevance(query, irrelevant_document)}")


relevant	(id 522843)	=	0.5448891448315577
irrelevant	(id 522983)	=	0.375


Полученные модели можно сохранять и переиспользовать без обучения

In [21]:
# Обратите внимание: у моделей уже реализованы to_json() и from_json(str) методы
# поэтому достаточно сохранить одну json-строку
with open("./vk_sdbn_model.json", "w", encoding="utf-8") as f_out:
    f_out.write(vk_model.to_json())

In [22]:
vk_model = SDBN()

# прочесть также достаточно только одну строку
with open("./vk_sdbn_model.json", "r", encoding="utf-8") as f_in:
    for line in f_in:
        vk_model.from_json(line)
        break